In [17]:

import numpy as np
import tensorflow as tf

#from keras.datasets import mnist
import plotly.express as px
from IPython.display import display

from scipy import sparse
from skimage.measure import block_reduce 

RuntimeError: module compiled against API version 0xf but this version of numpy is 0xe

ImportError: initialization failed

In [20]:
!pip uninstall tensorflow

^C


In [2]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data(path="mnist.npz") #mnist.load_data()
assert x_train.shape == (60000, 28, 28)
assert x_test.shape == (10000, 28, 28)
assert y_train.shape == (60000,)
assert y_test.shape == (10000,)

NameError: name 'tf' is not defined

In [ ]:
# Get all the values whos true classification is less than 5
y_train_i = np.nonzero(y_train < 5)[0]
# Find those grids in x_train
#display(x_train.shape)
x_train = x_train[y_train_i]
print(x_train.shape)

(30596, 28, 28)


In [ ]:
display(example[0])

array([  0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
        16.  , 130.  , 168.5 , 169.  , 168.5 ,  63.  ,   0.  ,   0.  ,
         0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  62.75, 233.5 , 201.75,
       119.5 ,  92.5 , 224.75, 221.  ,   0.  ,   0.  ,   0.  ,   0.  ,
         0.  ,   2.5 , 113.  , 244.  ,  93.5 ,   0.  ,   0.  ,   0.  ,
       130.25, 252.5 ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,  84.5 ,
       252.5 , 180.75,  11.75,   0.  ,   0.  ,   0.  , 191.75, 165.75,
         0.  ,   0.  ,   0.  ,   0.  ,  84.25, 246.25, 202.25,  21.75,
         0.  ,   0.  ,   0.  ,  28.5 , 231.5 ,  36.25,   0.  ,   0.  ,
      

In [ ]:
display(px.imshow(x_train[0], color_continuous_scale='gray'))

def image_compression(mat):
    n = mat.shape[0] # 28
    new = np.zeros(shape = (int(n/2), int(n/2)))
    display(mat.shape)
    for i in range(0, n, 2):
        display(mat[np.ix_([i, i+1], [i, i+1])])
        val = np.mean(mat[np.ix_([i, i+1], [i, i+1])])
        new[i//2, i//2] = val
        #for j in range(i, i+):

    # flatten the matrix by stacking columns 
    #new = new.flatten(order = "F")
    return new


def image_comp(data, flatten = False):
    ### Can compress image either into 14x14 or 196x1
    ### Regardless, scaled by 255
    if flatten == True:
        new_red = np.zeros((data.shape[0], 196))
        for i in range(data.shape[0]):
            new_red[i]  = block_reduce(data[i, :, :], block_size = (2,2), func = np.mean).flatten(order = "F")/255
    else:
        new_red = np.zeros((data.shape[0], 14, 14))
        for i in range(data.shape[0]):
            new_red[i]  = block_reduce(data[i, :, :], block_size = (2,2), func = np.mean)/255

    return new_red
    
x_train_comp = image_comp(x_train, flatten = True)
#px.imshow(example[0])


# Problem 3

In [ ]:
# We have five clusters
# We do not know which image it belongs to (K-means) but EM can help us figure it out by using mixture of gaussians
import gc
gc.collect()
#import silence_tensorflow.auto

#import tensorflow_probability as tfp 

from tqdm import tqdm # progress meter

def mlt_norm(x, mu, Sigma, log = True):
    # Dimensions ( not)
    k = len(mu)  
    # Find the constant
    constant = 1.0 / (np.sqrt((2 * np.pi) ** k * np.linalg.det(Sigma)))
    # Exponent term
    exponent = -0.5 * np.dot(np.dot((x - mu).T, np.linalg.inv(Sigma)), (x - mu))
    likelihood = constant * np.exp(exponent)

    # a little if-else for the log 
    if log == True:
        return np.log(likelihood)
    else:
        return likelihood


def em(data, n_classes, n_initializations, n_iterations, random_seed = 1001, sigma = "spherical", tol = 0.0001):
    #Set seed for reproducible results
    np.random.seed(random_seed)
    n_samples = data.shape[0] # Should be 30596 
    

    for initializations in range(n_initializations):
        # We estabilish our parameters with random generation, n_classes = clusters 
        # We initialize our parmaeters
        # Mu will be random sample of 5 rows from the 30,000+ rows, could do K-means but defeats the purpose
        mu = data[np.random.randint(data.shape[1], size = 5), :] #5 x 196
        
        if sigma == "spherical":
            # For the spherical case, we can take the average of all the variances of the 196 columns in the data, and apply that to an identity matrix
            var_avg = np.mean(np.var(data, axis = 1))
            Sigma = var_avg * np.eye(data.shape[1]) # Covariance matrix of the entire data set
            Sigma = np.repeat(Sigma[:, :, np.newaxis], n_classes, axis = 2)
            #print(Sigma[:,:,0])
        if sigma == "diagonal":
            # For the spherical case, we can take the average of all the variances of the 196 columns in the data, and apply that to an identity matrix
            var_avg = np.mean(np.var(data, axis = 1))
            Sigma = var_avg * np.eye(data.shape[1]) # Covariance matrix of the entire data set

        # For pi, we can use Dirichlet since it sums to 1
        pi = np.random.dirichlet(np.ones(n_classes))


        for iteration in range(n_iterations):
            # Declare likelihoods
            new_likelihood = -np.inf
            old_likelihood = -np.inf

            # E-Step
            Fij = np.zeros((n_samples, n_classes))
            ### To speed up complexity, we are going to try a tensor approach 
            ### We will do x - uj 5 times and save them into a tensor, this way we save ourselves redoing the calculation for each iteration
            tensor_calc = np.zeros((n_samples, data.shape[1], n_classes)) # 30596 x 196 x 5
            d = data.shape[1]
            for t in range(n_classes):
                x_mu = sparse.csr_matrix(data - mu[t,:]) # x - mu_j
                
                # First half 
                inv_sigma = sparse.csr_matrix(np.diag(1/np.diag(Sigma[:,:,t])))
                half = x_mu @ inv_sigma
                half = half.todense()
                product = np.einsum('ij,ji->i', half, x_mu.todense().T) # Only need the diagonal entries
                Fij[:,t] = pi[t]*np.log(1.0 / (np.sqrt((2 * np.pi) ** d * Sigma[:,:,t].diagonal().prod()))) + -0.5 *product 
                
            # To normalize, we divide each entry in a row of Fij by its row sum
            for j_prime in range(Fij.shape[0]):
                Fij[j_prime, :] /= Fij[j_prime, :].sum(keepdims = True)

            #print("Fij after normalization", Fij)

            # M step
            # Updating rules
            # pi and mu can be done outside of a loop
            pi = np.mean(Fij, axis = 0)
            mu = Fij.T @ data / np.sum(Fij, axis =0).reshape(-1,1)
            for col in range(n_classes):
                # Updating rules       
                x_mu = data - mu[col,:]
                prod = (Fij[:,col] @ (x_mu) @ (x_mu).T)
                #print("My prod is ", prod)
                Sigma[:,:, col] = prod.sum() / Fij[:,col].sum() 
                Sigma[:,:,col] = np.fill_diagonal(Sigma[:,:,col], Sigma[:,:,col].diagonal() + 0.05) # per hint provided 
            new_likelihood = mlt_norm(data[0,:], mu[0,:], Sigma[:,:,0])
            #print(new_likelihood)

            if np.abs(new_likelihood - old_likelihood)/old_likelihood < tol:
                break
            print("My new Sigma is, ", Sigma)
                
    return mu, Sigma, pi
    
em(x_train_comp, 5, 3, 10)

My prod is  [ 5.37478993e-11 -6.56227665e-11 -5.72782856e-11 ... -2.21619151e-11
 -8.62300598e-11 -3.97590740e-11]
My prod is  [ 7.50878080e-11 -1.25388044e-10 -3.97007662e-11 ... -7.78764092e-11
 -1.74742331e-10  7.77902229e-13]
My prod is  [ 4.45215333e-11 -3.78038291e-11 -3.90955680e-11 ...  3.19114729e-11
 -4.18238582e-11 -1.52268175e-11]
My prod is  [ 5.42098629e-11 -6.96892100e-11 -8.43377061e-12 ... -3.67575599e-11
 -8.07649575e-11  2.25271202e-11]
My prod is  [ 3.20320865e-11  4.10801720e-12 -2.64095892e-11 ...  6.15643319e-11
 -8.61118363e-12 -5.38006516e-11]


c:\Users\Gianni\anaconda3\lib\site-packages\numpy\linalg\linalg.py:2154: RuntimeWarning:

invalid value encountered in det



nan
My new Sigma is,  [[[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  ...
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  ...
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  ...
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 ...

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  ...
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  ...
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]

 [[nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  ...
  [nan nan nan nan nan]
  [nan nan nan nan nan]
  [nan nan nan nan nan]]]
My prod is  [nan nan nan ... nan nan nan]
My prod is  [nan n

(array([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan, nan, nan, nan, nan,